In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option("display.max_columns", None)
data = pd.read_csv("Data/preprocessed_data.csv", delimiter=",")
data.head(20)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,Admission grade bins,Age group,Unemployment rate bins
0,1,17,5,171,1,1,122.0,1.0,19,12,5,9,127.3,1,0,0,1,1.0,0,20.0,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout,NaN,18-22,10-15%
1,1,15,1,9254,1,1,160.0,1.0,1,3,3,3,142.5,1,0,0,0,1.0,0,19.0,0,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate,NaN,18-22,10-15%
2,1,1,5,9070,1,1,122.0,1.0,37,37,9,9,124.8,1,0,0,0,1.0,0,19.0,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout,NaN,18-22,10-15%
3,1,17,2,9773,1,1,122.0,1.0,38,37,5,3,119.6,1,0,0,1,0.0,0,20.0,0,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate,NaN,18-22,5-10%
4,2,39,1,8014,0,1,100.0,1.0,37,38,9,9,141.5,0,0,0,1,0.0,0,34.0,0,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate,NaN,28-35,10-15%
5,2,39,1,9991,0,19,133.1,1.0,37,37,9,7,114.8,0,0,1,1,1.0,0,34.0,0,0,5,10,5,11.857143,0,0,5,17,5,11.500000,5,16.2,0.3,-0.92,Graduate,NaN,28-35,15%+
6,1,1,1,9500,1,1,142.0,1.0,19,38,7,10,128.4,1,0,0,1,0.0,1,18.0,0,0,7,9,7,13.300000,0,0,8,8,8,14.345000,0,15.5,2.8,-4.06,Graduate,NaN,18-22,15%+
7,1,18,4,9254,1,1,119.0,1.0,37,37,9,9,113.1,1,0,0,0,1.0,0,22.0,0,0,5,5,0,0.000000,0,0,5,5,0,0.000000,0,15.5,2.8,-4.06,Dropout,NaN,18-22,15%+
8,1,1,3,9238,1,1,137.0,62.0,1,1,9,9,129.3,0,0,0,1,0.0,1,21.0,1,0,6,8,6,13.875000,0,0,6,7,6,14.142857,0,16.2,0.3,-0.92,Graduate,NaN,18-22,15%+
9,1,1,1,9238,1,1,138.0,1.0,1,19,4,7,123.0,1,0,1,0,0.0,0,18.0,0,0,6,9,5,11.400000,0,0,6,14,2,13.500000,0,8.9,1.4,3.51,Dropout,NaN,18-22,5-10%


In [3]:
# Preprocessing

# Encoding only categorical columns
categorical_columns = [
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance', 'Previous qualification', 'Nacionality',
    "Mother's qualification", "Father's qualification", "Mother's occupation",
    "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor',
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International', 'Target',
    'Age group', 'Unemployment rate bins'
]

for col in categorical_columns:
    data[col] = data[col].astype(str)

# One hot encoding
encoded_data = pd.get_dummies(data)

# Dropping all non categorical columns
encoded_data = encoded_data.drop(columns=["Previous qualification (grade)", "Admission grade",
                           "Age at enrollment", "Curricular units 1st sem (credited)",
                           "Curricular units 1st sem (enrolled)", "Curricular units 1st sem (evaluations)",
                           "Curricular units 1st sem (approved)", "Curricular units 1st sem (grade)",
                           "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)",
                           "Curricular units 2nd sem (enrolled)", "Curricular units 2nd sem (evaluations)",
                           "Curricular units 2nd sem (approved)", "Curricular units 2nd sem (grade)",
                           "Curricular units 2nd sem (without evaluations)", "Unemployment rate",
                           "Inflation rate", "GDP"])

encoded_data.head(20)

,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_10,Application mode_15,Application mode_16,Application mode_17,Application mode_18,Application mode_2,Application mode_26,Application mode_27,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_5,Application mode_51,Application mode_53,Application mode_57,Application mode_7,Application order_0,Application order_1,Application order_2,Application order_3,Application order_4,Application order_5,Application order_6,Application order_9,Course_171,Course_33,Course_8014,Course_9003,Course_9070,Course_9085,Course_9119,Course_9130,Course_9147,Course_9238,Course_9254,Course_9500,Course_9556,Course_9670,Course_9773,Course_9853,Course_9991,Daytime/evening attendance_0,Daytime/evening attendance_1,Previous qualification_1,Previous qualification_10,Previous qualification_12,Previous qualification_14,Previous qualification_15,Previous qualification_19,Previous qualification_2,Previous qualification_3,Previous qualification_38,Previous qualification_39,Previous qualification_4,Previous qualification_40,Previous qualification_42,Previous qualification_43,Previous qualification_5,Previous qualification_6,Previous qualification_9,Nacionality_1.0,Nacionality_100.0,Nacionality_101.0,Nacionality_103.0,Nacionality_105.0,Nacionality_108.0,Nacionality_109.0,Nacionality_11.0,Nacionality_13.0,Nacionality_14.0,Nacionality_17.0,Nacionality_2.0,Nacionality_21.0,Nacionality_22.0,Nacionality_24.0,Nacionality_25.0,Nacionality_26.0,Nacionality_32.0,Nacionality_41.0,Nacionality_6.0,Nacionality_62.0,Nacionality_nan,Mother's qualification_1,Mother's qualification_10,Mother's qualification_11,Mother's qualification_12,Mother's qualification_14,Mother's qualification_18,Mother's qualification_19,Mother's qualification_2,Mother's qualification_22,Mother's qualification_26,Mother's qualification_27,Mother's qualification_29,Mother's qualification_3,Mother's qualification_30,Mother's qualification_34,Mother's qualification_35,Mother's qualification_36,Mother's qualification_37,Mother's qualification_38,Mother's qualification_39,Mother's qualification_4,Mother's qualification_40,Mother's qualification_41,Mother's qualification_42,Mother's qualification_43,Mother's qualification_44,Mother's qualification_5,Mother's qualification_6,Mother's qualification_9,Father's qualification_1,Father's qualification_10,Father's qualification_11,Father's qualification_12,Father's qualification_13,Father's qualification_14,Father's qualification_18,Father's qualification_19,Father's qualification_2,Father's qualification_20,Father's qualification_22,Father's qualification_25,Father's qualification_26,Father's qualification_27,Father's qualification_29,Father's qualification_3,Father's qualification_30,Father's qualification_31,Father's qualification_33,Father's qualification_34,Father's qualification_35,Father's qualification_36,Father's qualification_37,Father's qualification_38,Father's qualification_39,Father's qualification_4,Father's qualification_40,Father's qualification_41,Father's qualification_42,Father's qualification_43,Father's qualification_44,Father's qualification_5,Father's qualification_6,Father's qualification_9,Mother's occupation_0,Mother's occupation_1,Mother's occupation_10,Mother's occupation_122,Mother's occupation_123,Mother's occupation_125,Mother's occupation_131,Mother's occupation_132,Mother's occupation_134,Mother's occupation_141,Mother's occupation_143,Mother's occupation_144,Mother's occupation_151,Mother's occupation_152,Mother's occupation_153,Mother's occupation_171,Mother's occupation_173,Mother's occupation_175,Mother's occupation_191,Mother's occupation_192,Mother's occupation_193,Mother's occupation_194,Mother's occupation_2,Mother's occupation_3,Mother's occupation_4,Mother's occupation_5,Mother's occupation_6,Mother's occupation_7,Mother's occupation_8,Mother's o

In [4]:
# improved version
import itertools

def build_inverted_index(data):
    inverted_index = {}
    for column in data:
        inverted_index[column] = set(data.index[data[column]])
    return inverted_index

support_cache = {}
def calculate_support(candidate, data, inverted_index):
    if candidate in support_cache:
        return support_cache[candidate]
    # Get common transactions using intersection of transactions from the inverted index
    common_transactions = set.intersection(*(inverted_index[item] for item in candidate))
    support_count = len(common_transactions)
    support_cache[candidate] = support_count
    return support_count

def apriori_itemsets(data, support, maxlen=10):
    inverted_index = build_inverted_index(data)
    min_support_count = support * data.shape[0]
    support_cache = {}
    
    # Step 0: Generate k=1 frequent itemsets
    frequent = []
    for column in data:
        itemset = frozenset([column])
        support_count = calculate_support(itemset, data, inverted_index)
        if support_count >= min_support_count:
            frequent.append(itemset)

    all_frequent = frequent.copy()
    k = 1

    # Main loop: Repeat until frequent is empty or k=maxlen
    while frequent and k < maxlen:
        k += 1
        candidates = []
        
        # Generate candidate k-itemsets from frequent (k-1)-itemsets
        for i in range(len(frequent)):
            for j in range(i+1, len(frequent)):
                candidate = frequent[i].union(frequent[j])
                if candidate not in candidates:
                    candidates.append(candidate)

        # Check support for each candidate
        frequent = []
        for candidate in candidates:
            support_count = calculate_support(candidate, data, inverted_index)
            if support_count >= min_support_count:
                frequent.append(candidate)

        all_frequent += frequent

    return all_frequent

In [121]:
frequent_itemsets = apriori_itemsets(encoded_data, support=0.05, maxlen=2)
print(frequent_itemsets[:1000])

[frozenset({'Marital status_1'}), frozenset({'Marital status_2'}), frozenset({'Application mode_1'}), frozenset({'Application mode_17'}), frozenset({'Application mode_39'}), frozenset({'Application mode_43'}), frozenset({'Application order_1'}), frozenset({'Application order_2'}), frozenset({'Application order_3'}), frozenset({'Application order_4'}), frozenset({'Course_9070'}), frozenset({'Course_9085'}), frozenset({'Course_9147'}), frozenset({'Course_9238'}), frozenset({'Course_9254'}), frozenset({'Course_9500'}), frozenset({'Course_9670'}), frozenset({'Course_9773'}), frozenset({'Course_9991'}), frozenset({'Daytime/evening attendance_0'}), frozenset({'Daytime/evening attendance_1'}), frozenset({'Previous qualification_1'}), frozenset({'Nacionality_1.0'}), frozenset({"Mother's qualification_1"}), frozenset({"Mother's qualification_19"}), frozenset({"Mother's qualification_3"}), frozenset({"Mother's qualification_37"}), frozenset({"Mother's qualification_38"}), frozenset({"Father's qu

In [5]:
def AprioriRules(data, frequent_itemsets, min_confidence):
    # association rules are stored in a list of dicts and then converted to a DataFrame at the end
    inverted_index = build_inverted_index(data)
    rules_list = []

    # loop through itemsets and generate all non-empty subsets
    for itemset in frequent_itemsets:
        subsets = []
        for length in range(1, len(itemset)):
            # generates all combinations of the subset of a certain length
            subsets.extend(itertools.combinations(itemset, length))
        sub_frozensets = [frozenset(subset) for subset in subsets]
        
        # Steps for generating rules
        itemset_support_count = calculate_support(itemset, data, inverted_index)  # Fetch precomputed support of the itemset
        
        for subset in sub_frozensets:
            # Define antecedent as the subset
            antecedent = subset
            # Define consequent as the itemset - the subset
            consequent = itemset.difference(subset)
            
            # Fetch precomputed support of the subset
            subset_support_count = calculate_support(subset, data, inverted_index)
            
            # Calculate confidence
            confidence = itemset_support_count / subset_support_count
            
            # Check if confidence meets minimum threshold and store rule
            if confidence >= min_confidence:
                rules_list.append({
                    "antecedent": antecedent,
                    "consequent": consequent,
                    "confidence": confidence
                })

    # Convert the list of rules to a DataFrame after all rules are generated
    rules = pd.DataFrame(rules_list, columns=["antecedent", "consequent", "confidence"])
    
    return rules

In [123]:
rules = AprioriRules(encoded_data, frequent_itemsets, min_confidence=0.7)
rules.head(10)

,antecedent,consequent,confidence
0,(Application mode_1),(Marital status_1),0.978723
1,(Application mode_17),(Marital status_1),0.992665
2,(Application mode_43),(Marital status_1),0.820946
3,(Application order_1),(Marital status_1),0.848538
4,(Application order_2),(Marital status_1),0.937378
5,(Application order_3),(Marital status_1),0.982639
6,(Application order_4),(Marital status_1),0.978992
7,(Course_9085),(Marital status_1),0.945161
8,(Course_9147),(Marital status_1),0.884507
9,(Course_9238),(Marital status_1),0.940476


In [124]:
sorted_rules = rules.sort_values(by='confidence', ascending=False)
sorted_rules.head(10)

,antecedent,consequent,confidence
182,(Course_9147),(Daytime/evening attendance_1),1.0
241,(Daytime/evening attendance_0),(Educational special needs_0),1.0
170,(Course_9070),(Daytime/evening attendance_1),1.0
172,(Course_9085),(Daytime/evening attendance_1),1.0
200,(Course_9254),(Daytime/evening attendance_1),1.0
203,(Course_9254),(Educational special needs_0),1.0
206,(Course_9500),(Daytime/evening attendance_1),1.0
216,(Course_9670),(Daytime/evening attendance_1),1.0
223,(Course_9773),(Daytime/evening attendance_1),1.0
231,(Course_9991),(Daytime/evening attendance_0),1.0


In [15]:
import time

start_time = time.time()
frequent_itemsets = apriori_itemsets(encoded_data, support=0.50, maxlen=2)
rules = AprioriRules(encoded_data, frequent_itemsets, min_confidence=0.90)
print("--- %s seconds ---" % (time.time() - start_time))

sorted_rules = rules.sort_values(by='confidence', ascending=False)
sorted_rules.to_csv("Data/found_rules_50_90_2S.csv", index=False)

--- 0.0431208610534668 seconds ---
